In [10]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import sys
sys.path.insert(0, '../../../../')

import numpy as np
import matplotlib.pyplot as plt
import pickle

from src.difsched.config import getExpConfig, visualizeExpConfig
from src.difsched.config import getEnvConfig, visualizeEnvConfig
from src.difsched.config import getDatasetConfig, visualizeDatasetConfig
from src.difsched.env.EnvironmentSim import createEnv
from src.difsched.utils.EnvInterface import EnvInterface
from src.difsched.training import training

In [11]:
hyperparams = {
    'N_diffusion_steps':30,
    'schedule_type': "vp",
    'abs_action_max': 1.0,
    'gamma': 0.99,
    'lr': 5e-4,
    'decay_lr': True,
    'weight_decay': 0.0,
    'num_critics': 8,
    'lcb_coef': 0.15,
    'q_sample_eta': 1.0,
    'weight_entropy_loss': 0.01,
    'weight_q_loss': 1.0,
    'approximate_action': True,
    'ema_tau': 0.001,
    'ema_period': 20,
    'ema_begin_update': 1000,
    'layer_norm': True,
    'grad_clip': 3.0,
    'device': 'cuda',
}

WithoutBchyperparams = {
    'N_diffusion_steps':30,
    'schedule_type': "vp",
    'abs_action_max': 1.0,
    'gamma': 0.99,
    'lr': 5e-3,
    'decay_lr': True,
    'weight_decay': 0.0,
    'num_critics': 8,
    'lcb_coef': 0.01,
    'q_sample_eta': 1.0,
    'weight_entropy_loss': 0.01,
    'weight_q_loss': 1.0,
    'approximate_action': True,
    'ema_tau': 0.001,
    'ema_period': 20,
    'ema_begin_update': 1000,
    'layer_norm': True,
    'grad_clip': 3.0,
    'device': 'cuda',
}

trainingConfig = {
    'BC_loss': True,
    'iterations': 100,
    'batch_size': 100,
    'LEN_eval': 50,
    'report_period': 10,
    'len_period': 50,
    'warm_up_period': 50,
    'max_sp_ratio': 1.0,
    'min_sp_ratio': 0.5,
    'max_weight_bc_loss': 1.0,
    'min_weight_bc_loss': 1.0,
    'rb_capacity': 30000
}

trainingWithoutBcConfig = {
    'BC_loss': False,
    'iterations': 100,
    'batch_size': 100,
    'LEN_eval': 50,
    'report_period': 10,
    'len_period': 50,
    'warm_up_period': 50,
    'max_sp_ratio': 0.01,
    'min_sp_ratio': 0.01,
    'max_weight_bc_loss': 1.0,
    'min_weight_bc_loss': 1.0,
    'rb_capacity': 30000
}

In [12]:
for expConfigIdx in range(4):
    expParams = getExpConfig(expConfigIdx)
    visualizeExpConfig(expParams)

    envConfigIdx = expParams['env_config_idx']
    envParams = getEnvConfig(envConfigIdx)
    visualizeEnvConfig(envParams)

    datasetConfigIdx = expParams['dataset_train_config_idx']
    datasetParams = getDatasetConfig(datasetConfigIdx)
    visualizeDatasetConfig(datasetParams)

    trafficDataParentPath = f'../../../../data/processed/traffic'
    env = createEnv(envParams, trafficDataParentPath)
    env.selectMode(mode="train", type="data")
    envInterface = EnvInterface(envParams, discrete_state=False)

    datasetFolder = f'../../../../data/processed/offline_dataset'
    with open(f'{datasetFolder}/subOptimalAgent_encConfig{datasetConfigIdx}_{envParams["sub_agents_idx"]}.pkl', 'rb') as f:
        dataset_expert = pickle.load(f)
    dataset_off = {
        'observations': dataset_expert['uRecord'],     
        'actions': dataset_expert['actionsRecord'], 
        'rewards': dataset_expert['rewardRecord'], 
        'next_observations': dataset_expert['uNextRecord']
    }
    print(f"Avg. packet loss rate: {np.mean(dataset_expert['rewardRecord'])}")
    print(f"length of dataset: {len(dataset_off['observations'])}")

    # With BC
    
    save_folder = f"../../../../data/results/dql/config_{expConfigIdx}"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    training(
        trainingConfig, dataset_off, hyperparams, env, envInterface, save_folder, N_exp_list=[1,2]
    )
    

    # Without BC
    save_folder = f"../../../../data/results/dql/config_{expConfigIdx}/without_bc"
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    training(
        trainingWithoutBcConfig, dataset_off, WithoutBchyperparams, env, envInterface, save_folder, N_exp_list=[1,2]
    )

dataset_train_config_idx: 0
dataset_test_config_idx: 2
env_config_idx: 0
Environment Configuration
Number of Users:        8
Window Length:          200
Dataflow:               thumb_fr
Sigmoid K List:         [0.1, 0.2, 0.3, 0.4, 0.5]
Sigmoid S List:         [10.0, 10.0, 10.0, 10.0, 10.0]
Resource Bar:           5
Bandwidth:              100
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
Dataset Configuration
Number of Users:        8
Window Length:          200
N_aggregation:          4
Dataflow:               thumb_fr
Random Seed:            999
Resource Bar:           5
Bandwidth:              100
Sigmoid K List:         [0.3]
Sigmoid S List:         [10.0]
Sub Agents:             [[0, 0]]
User Map:               [[0, 1, 2, 3], [4, 5, 6, 7]]
Avg. packet loss rate: 0.32451573558442404
length of dataset: 10000
1.0 1.0
Expert's Reward: 0.6756755709648132
state_dim: 8, action_dim: 8


====================Iteration 10====================
Ld: 0.14715328589081764, Lq: 0.6695501345396042, Le: -1.5933293879032135, loss_Q: 0.00813582656905055
Avg. Reward: 0.4141829729853999, sample_ratio: 0.9, weight_bc_loss: 1.0


save model 1_best, smoothed reward: 0.393937292974722


save model 1_best, smoothed reward: 0.3869814588384461


save model 1_best, smoothed reward: 0.38185780598789687


...

save model 2_best, smoothed reward: 0.7818874744575437


====================Iteration 20====================
Ld: 0.6614537131786347, Lq: 0.21432434067130088, Le: -1.4481236815452576, loss_Q: 0.011510680122300982
Avg. Reward: 0.7945696607189896, sample_ratio: 0.01, weight_bc_loss: 1.0


====================Iteration 30====================
Ld: 0.6083961999416352, Lq: 0.22709284350275993, Le: -1.4512690091133118, loss_Q: 0.006044489000923931
Avg. Reward: 0.7904928675751094, sample_ratio: 0.01, weight_bc_loss: 1.0


save model 2_best, smoothed reward: 0.7788381438283044
====================Iteration 40====================
Ld: 0.591794530749321, Lq: 0.22064969539642335, Le: -1.4607233834266662, loss_Q: 0.008998885001055897
Avg. Reward: 0.7845525738993928, sample_ratio: 0.01, weight_bc_loss: 1.0


====================Iteration 50====================
Ld: 0.5586224341392517, Lq: 0.23562367022037506, Le: -1.457375227212906, loss_Q: 0.0040179467387497425
Avg. Reward: 0.7903630243824957, sample_ratio: 0.01, weight_bc_loss: 1.0
